In [1]:
import sys
sys.path.append("/opt/drake/lib/python3.10/site-packages")  # The jammy dockerfile needs this.

import numpy as np
from pydrake.all import RobotDiagramBuilder, HybridMultibodyCollocation, Solve, eq
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DirectCollocation,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    FiniteHorizonLinearQuadraticRegulator,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PlanarSceneGraphVisualizer,
    SceneGraph,
    Simulator,
    Solve,
    TrajectorySource,
)

from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat,
                         MultibodyPlant, InitializeAutoDiffTuple, ExtractGradient, TrajectorySource,
                         FixedOffsetFrame, QuaternionFloatingJoint, UniformGravityFieldElement,
                         ExternallyAppliedSpatialForce, AbstractValue, ContactModel, GeometrySet,
                         CollisionFilterDeclaration, ContactResults, SpatialAcceleration, SpatialForce,
                         SceneGraph, DiagramBuilder_, AutoDiffXd)

from pydrake.common import temp_directory
from pydrake.geometry import (
    MeshcatVisualizerParams,
    Role,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.systems.framework import BasicVector, LeafSystem
import pydrake.symbolic as sym
import pydrake.autodiffutils as autodiff
from pydrake.trajectories import PiecewisePolynomial

np.set_printoptions(suppress=True, precision=3)

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()
meshcat.Set2dRenderMode()

INFO:drake:Meshcat listening for connections at https://067a6c9a-b930-4c28-8084-e86f86716a80.deepnoteproject.com/7000/


## URDFs

In [3]:
climber = """
<?xml version="1.0"?><robot name="Climber">
  <link name="world">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 0" />
      </material>
    </visual>
  </link>

  <link name="hand">
    <inertial>
      <origin xyz="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="1 0.647 0 1" />
      </material>
    </visual>

    <collision>
      <geometry>
        <sphere radius=".05" />
      </geometry>
    </collision>
  </link>

  <link name="arm0">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm1">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm2">
    <inertial>
      <origin xyz="0 0 -1" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -1" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="1"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <joint name="world-hand" type="planar">
    <parent link="world" />
    <child link="hand" />
    <axis xyz="0 1 0" />
  </joint>
  
  <joint name="theta0" type="continuous">
    <parent link="hand" />
    <child link="arm0" />
    <axis xyz="0 -1 0" />
  </joint>

  <joint name="theta1" type="continuous">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm0" />
    <child link="arm1" />
    <axis xyz="0 -1 0" />
  </joint>

  <joint name="theta2" type="continuous">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm1" />
    <child link="arm2" />
    <axis xyz="0 -1 0" />
  </joint>
  
  <transmission type="SimpleTransmission" name="theta1_force">
    <actuator name="force1" />
    <joint name="theta1" />
  </transmission>

  <transmission type="SimpleTransmission" name="theta2_force">
    <actuator name="force2" />
    <joint name="theta2" />
  </transmission>
  
  </robot>
</xml>
"""

In [4]:
obstacle_background = """
<?xml version="1.0"?><robot name="Obstacle_Background">
  <link name="obstacle_base_background">
    <visual>
      <origin xyz="0.125 200 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 1 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>
  </link>

  <link name="obstacle_arms_background">
    <visual>
      <origin xyz="0 200 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>
  </link>
  
  <joint name="obstacle_base_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base_background" />
    <child link="obstacle_arms_background" />
    <axis xyz="0 -1 0" />
  </joint>
  </robot>
</xml>
"""

In [5]:
obstacle = """
<?xml version="1.0"?><robot name="Obstacle">
  <link name="obstacle_base">
    <visual>
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision name="obstacle_base_collision">
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
    </collision>
  </link>

  <link name="obstacle_arms">
    <visual>
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision name="obstacle_top_arm">
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>

    <collision name="obstacle_middle_arm">
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>
  </link>
  
  <joint name="obstacle_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base" />
    <child link="obstacle_arms" />
    <axis xyz="0 -1 0" />
  </joint>

  </robot>
</xml>
"""

## Testing Stuff

In [7]:
# visualizer = ModelVisualizer(meshcat=meshcat)
# visualizer.parser().AddModelsFromString(climber, "urdf")
# visualizer.parser().AddModelsFromString(obstacle_background, "urdf")
# visualizer.parser().AddModelsFromString(obstacle, "urdf")
# visualizer.Run(loop_once=False)
# meshcat.Delete()
# meshcat.DeleteAddedControls()

## Solve

In [8]:
def HybridClimber():
    builder = RobotDiagramBuilder()
    
    # Load the climber 
    builder.parser().AddModelsFromString(climber, "urdf")

    # Load the obstacle
    builder.parser().AddModelsFromString(obstacle_background, "urdf")
    builder.parser().AddModelsFromString(obstacle, "urdf")

    plant = builder.plant()
    scene_graph = builder.scene_graph()

    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = diagram.plant().GetMyContextFromRoot(context)

    # ['world-hand_x', 'world-hand_y', 'world-hand_qz', 'theta0_q', 'theta1_q', 'theta2_q', 
    #  'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 'theta0_w', 'theta1_w', 'theta2_w']
    # NOTE: world-hand_x is Z, world-hand_y is X
    plant.SetPositions(plant_context, np.array([0.1, 0.02, 0., 0., 0., 0.]))
    plant.SetVelocities(plant_context, np.array([0., 0., 0., 0., 0., 0.]))

    print(plant.GetStateNames())
    inspector = scene_graph.model_inspector()

    kMinTimeStep = 0.01
    kMaxTimeStep = 0.1
    hybrid = HybridMultibodyCollocation(diagram, context, kMinTimeStep,
                                    kMaxTimeStep)
    prog = hybrid.prog()

    pair_contacts = hybrid.GetContactPairCandidates()
    print(pair_contacts)
    print(len(pair_contacts))

    contacts = [None, None, None] # middle bar, top bar, ladder back
    for contact in pair_contacts:
        id1, id2 = contact
        print(inspector.GetName(id1), inspector.GetName(id2))
        contact_name = inspector.GetName(id2)
        if (contact_name == "Obstacle::obstacle_middle_arm"):
            contacts[0] = contact
        if (contact_name == "Obstacle::obstacle_top_arm"):
            contacts[1] = contact
        if (contact_name == "Obstacle::obstacle_base_collision"):
            contacts[2] = contact

    # initially falling onto first bar
    kNumTimeSteps_aerial_init = 5
    aerial_init = hybrid.AddMode("aerial_init", kNumTimeSteps_aerial_init, set())
    aerial_init.AddEqualTimeIntervalsConstraints()

    initial_state = [0.1, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(initial_state, initial_state, aerial_init.initial_state())

    u_aerial_init = aerial_init.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR AERIAL INIT


    # catching middle bar, sliding down to hit back
    kNumTimeSteps_middle_bar_contact = 2
    middle_bar_contact = hybrid.AddModeWithInelasticImpact(
        "middle_bar_contact", kNumTimeSteps_middle_bar_contact, contacts[0])
    middle_bar_contact.AddEqualTimeIntervalsConstraints()

    u_middle_bar_contact = middle_bar_contact.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR MIDDLE BAR CONTACT
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[0] >= 0)


    # hitting back of ladder initially
    kNumTimeSteps_middle_back_bar_contact = 2
    middle_back_bar_contact = hybrid.AddModeWithInelasticImpact(
        "middle_back_bar_contact", kNumTimeSteps_middle_back_bar_contact, contacts[2])
    middle_back_bar_contact.AddEqualTimeIntervalsConstraints()

    u_middle_back_bar_contact = middle_back_bar_contact.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR MIDDLE-BACK BAR CONTACT
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[0] >= 0)


    # hitting back of ladder, turn into joint
    kNumTimeSteps_middle_bar_grab = 200
    middle_bar_grab = hybrid.AddMode(
        "middle_bar_grab", kNumTimeSteps_middle_bar_grab, {contacts[0], contacts[2]})
    middle_bar_grab.AddEqualTimeIntervalsConstraints()

    u_middle_bar_grab = middle_bar_grab.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR MIDDLE BAR GRAB
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[0] >= 0)
    

    # let go, launching robot up
    kNumTimeSteps_aerial_launch_1 = 50
    aerial_launch_1 = hybrid.AddMode("aerial_launch_1", kNumTimeSteps_aerial_launch_1, set())
    aerial_launch_1.AddEqualTimeIntervalsConstraints()

    u_aerial_launch_1 = aerial_launch_1.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 1
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[0] >= 0.09)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[0] <= 0.39)

    
    # let go, launching robot up
    kNumTimeSteps_aerial_launch_2 = 50
    aerial_launch_2 = hybrid.AddMode("aerial_launch_2", kNumTimeSteps_aerial_launch_2, set())
    aerial_launch_2.AddEqualTimeIntervalsConstraints()

    u_aerial_launch_2 = aerial_launch_2.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 2
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[1] <= -0.19)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[1] >= -0.5)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[0] >= 0.09)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[0] <= 1)

    
    # let go, launching robot up
    kNumTimeSteps_aerial_launch_3 = 50
    aerial_launch_3 = hybrid.AddMode("aerial_launch_3", kNumTimeSteps_aerial_launch_3, set())
    aerial_launch_3.AddEqualTimeIntervalsConstraints()

    # ends in position right above top bar
    final_state = [0.6, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(final_state, final_state, aerial_launch_3.final_state())

    u_aerial_launch_3 = aerial_launch_3.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 3
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[0] >= 0.59)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[0] <= 1)

    
    # catching top bar, sliding down to hit back
    kNumTimeSteps_top_bar_contact = 2
    top_bar_contact = hybrid.AddModeWithInelasticImpact(
        "top_bar_contact", kNumTimeSteps_top_bar_contact, contacts[1])
    top_bar_contact.AddEqualTimeIntervalsConstraints()
    
    u_top_bar_contact = top_bar_contact.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR TOP BAR CONTACT
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[0] >= 0.5)


    # hitting back of ladder initially
    kNumTimeSteps_top_back_bar_contact = 2
    top_back_bar_contact = hybrid.AddModeWithInelasticImpact(
        "top_back_bar_contact", kNumTimeSteps_top_back_bar_contact, contacts[2])
    top_back_bar_contact.AddEqualTimeIntervalsConstraints()

    u_top_back_bar_contact = top_back_bar_contact.input()

    # TODO: ADD COSTS AND CONSTRAINTS FOR TOP-BACK BAR CONTACT
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[0] >= 0.5)


    # hitting back of ladder, turn into joint
    kNumTimeSteps_top_bar_grab = 2
    top_bar_grab = hybrid.AddMode(
        "top_bar_grab", kNumTimeSteps_top_bar_grab, {contacts[1], contacts[2]})
    top_bar_grab.AddEqualTimeIntervalsConstraints()
    
    u_top_bar_grab = top_bar_grab.input()
    
    # TODO: ADD COSTS AND CONSTRAINTS FOR TOP BAR GRAB
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[0] >= 0.5)


    result = Solve(prog)
                        
    traj = hybrid.ReconstructStateTrajectory(result)
    force = hybrid.ReconstructContactForceTrajectory(
        result, hybrid.GetContactPairCandidates().pop())

    print('all phases [time, state, contact force]')
    print(np.vstack((traj.get_segment_times(), 
                    traj.vector_values(traj.get_segment_times()),
                    force.vector_values(traj.get_segment_times()))).T)
    
    x_trajectory = traj
    u_trajectory = hybrid.ReconstructInputTrajectory(result)

    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(prog))

    return traj, u_trajectory

x_trajectory, u_trajectory = HybridClimber()

['world-hand_x', 'world-hand_y', 'world-hand_qz', 'theta0_q', 'theta1_q', 'theta2_q', 'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 'theta0_w', 'theta1_w', 'theta2_w']
{(<GeometryId value=171>, <GeometryId value=226>), (<GeometryId value=171>, <GeometryId value=229>), (<GeometryId value=171>, <GeometryId value=213>)}
3
Climber::Sphere Obstacle::obstacle_top_arm
Climber::Sphere Obstacle::obstacle_middle_arm
Climber::Sphere Obstacle::obstacle_base_collision
all phases [time, state, contact force]
[[ 0.     0.1    0.02  ...  0.     0.     0.   ]
 [ 0.017  0.098  0.016 ...  0.     0.     0.   ]
 [ 0.035  0.089  0.006 ...  0.     0.     0.   ]
 ...
 [ 7.821  0.6    0.025 ... -0.769  0.    -4.695]
 [ 7.846  0.6    0.025 ... -0.892  0.    -3.746]
 [ 7.858  0.596  0.025 ...  0.     0.    -3.328]]
['aerial_init collocation constraint for segment 0[0]: 0.000000 <= 0.004411 <= 0.000000', 'aerial_init collocation constraint for segment 0[1]: 0.000000 <= -0.011814 <= 0.000000', 'aerial_init co

## Simulate

In [11]:
def create_scene(x_traj_source, sim_time_step=0.0001):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant = MultibodyPlant(time_step=0.0)
    scene_graph = SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)

    # Loading models.
    # Load the climber 
    parser.AddModelsFromString(climber, "urdf")

    # Load the obstacle
    parser.AddModelsFromString(obstacle_background, "urdf")
    parser.AddModelsFromString(obstacle, "urdf")

    #Weld the obstacle to the world so that it's fixed during the simulation.
    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

    # Finalize the plant after loading the scene.
    plant.Finalize()
    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    plant.SetPositions(plant_context, np.array([0.1, 0.02, 0., 0., 0., 0.]))
    plant.SetVelocities(plant_context, np.array([0., 0., 0., 0., 0., 0.]))

    builder.AddSystem(x_traj_source)
    builder.AddSystem(scene_graph)
    pos_to_pose = builder.AddSystem(
        MultibodyPositionToGeometryPose(plant, input_multibody_state=True)
    )

    builder.Connect(x_traj_source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(
        pos_to_pose.get_output_port(),
        scene_graph.get_source_pose_port(plant.get_source_id()),
    )

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()
    return diagram, visualizer

In [12]:
x_traj_source = TrajectorySource(x_trajectory)

def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(x_traj_source, sim_time_step):
    diagram, visualizer = create_scene(x_traj_source, sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(10.0)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(x_traj_source, sim_time_step=0.0001)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=067a6c9a-b930-4c28-8084-e86f86716a80' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>